In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Carga de datos
archivo = "../DEPURATION/datos_limpios.xlsx"  # Cambia al nombre de tu archivo
df = pd.read_excel(archivo)

# Preprocesamiento

# Crear variable binaria para accidente: 1 si TOTAL_VICTIMAS > 0, de lo contrario 0
df['accidente_ocurrido'] = (df['TOTAL_VICTIMAS'] > 0).astype(int)

# Seleccionar variables de interés
features = ['MES', 'DIA_MES', 'METEO_ORIGEN', 'METEO_DESTINO', 'provincia_origen', 'provincia_destino']
X = df[features]
y = df['accidente_ocurrido']


# Codificar variables categóricas
X_encoded = pd.get_dummies(X, columns=['METEO_ORIGEN', 'METEO_DESTINO', 'provincia_origen', 'provincia_destino'], drop_first=True)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Entrenar el modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidad de accidente

# Evaluación del modelo
print("Exactitud:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))
print("\nInforme de clasificación:\n", classification_report(y_test, y_pred))


Exactitud: 0.9734291094218565
AUC-ROC: 0.9965846036793008

Informe de clasificación:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     98563
           1       0.94      0.95      0.94     30488

    accuracy                           0.97    129051
   macro avg       0.96      0.96      0.96    129051
weighted avg       0.97      0.97      0.97    129051



In [2]:
import joblib
joblib.dump(model, 'modelo_random_forest_weights.pkl')
print("Modelo guardado como 'modelo_random_forest.pkl'")

Modelo guardado como 'modelo_random_forest.pkl'


In [4]:
model_cargado = joblib.load('../MODEL/modelo_random_forest_weights.pkl')
archivo = "../DEPURATION/datos_limpios.xlsx"  # Cambia al nombre de tu archivo

df = pd.read_excel(archivo)
df['accidente_ocurrido'] = (df['TOTAL_VICTIMAS'] > 0).astype(int)

X_v2 = df[features]
y_v2 = df['accidente_ocurrido']

X_v2_encoded = pd.get_dummies(X_v2, columns=['METEO_ORIGEN', 'METEO_DESTINO', 'provincia_origen', 'provincia_destino'], drop_first=True)
y_pred_nuevo = model_cargado.predict(X_v2_encoded)
y_pred_proba_nuevo = model_cargado.predict_proba(X_v2_encoded)[:, 1]

print("Exactitud:", accuracy_score(y_v2, y_pred_nuevo))
print("AUC-ROC:", roc_auc_score(y_v2, y_pred_proba_nuevo))
print("\nInforme de clasificación:\n", classification_report(y_v2, y_pred_nuevo))

Exactitud: 0.9946858136485787
AUC-ROC: 0.999633461635886

Informe de clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    493039
           1       0.99      0.99      0.99    152215

    accuracy                           0.99    645254
   macro avg       0.99      0.99      0.99    645254
weighted avg       0.99      0.99      0.99    645254



In [5]:
X_test_total = X_v2.loc[X_v2_encoded.index]  # Seleccionamos las filas originales correspondientes a X_test
resultado = X_test_total.copy()
resultado['PROBABILIDAD_DE_ACCIDENTE'] = y_pred_proba_nuevo
resultado.to_excel('final_predicciones_accidentes.xlsx', index=False)
resultado.head()


,MES,DIA_MES,METEO_ORIGEN,METEO_DESTINO,provincia_origen,provincia_destino,PROBABILIDAD_DE_ACCIDENTE
0,1,1,NaN,1.0,2,3,0.94
1,1,1,NaN,1.0,2,4,0.97
2,1,1,NaN,1.0,2,6,0.88
3,1,1,NaN,1.0,2,8,0.93
4,1,1,NaN,NaN,2,9,0.20
